## Hidden Markov Models — Viterbi Algorithm

The **Viterbi algorithm** is a dynamic programming method used to find the **most probable sequence of hidden states** (also called the *optimal path*) in a Hidden Markov Model (HMM), given a sequence of observed events.

### Mathematical formulation

Let:  
- $s_t$ = hidden state at time $t$  
- $o_t$ = observation at time $t$  
- $A_{ij} = P(s_t = j \mid s_{t-1} = i)$ = transition probability  
- $B_j(o_t) = P(o_t \mid s_t = j)$ = emission probability  
- $\pi_i = P(s_1 = i)$ = initial state distribution  

We define the **Viterbi variable**:

$$
\delta_t(j) = \max_{s_1, \dots, s_{t-1}} P(s_1, \dots, s_{t-1}, s_t = j, o_1, \dots, o_t)
$$

It represents the **probability of the most likely path** that ends in state $j$ after observing $o_1, \dots, o_t$.

---

### Recursive formulation

1. **Initialization**

$$
\delta_1(j) = \pi_j \, B_j(o_1)
$$

2. **Recursion**

$$
\delta_t(j) = B_j(o_t) \, \max_i \left[ \delta_{t-1}(i) \, A_{ij} \right]
$$

and we keep track of which previous state $i$ achieved this maximum:

$$
\psi_t(j) = \arg\max_i \left[ \delta_{t-1}(i) \, A_{ij} \right]
$$

3. **Termination**

$$
P^* = \max_j \delta_T(j)
$$

4. **Backtracking**

The most likely state sequence is obtained by recursively following the stored backpointers:

$$
s_T^* = \arg\max_j \delta_T(j), \quad s_{t-1}^* = \psi_t(s_t^*)
$$

---

### Intuition of the algo

At each time step, the algorithm computes — for every possible hidden state — the probability of **the most likely path that could have led there** given the observations so far.  
Instead of keeping track of *all possible paths*, it stores only:
- the **best probability** for each state (`viterbi_probs`), and  
- the **index of the previous state** that led to it (`backpointer`).

By the end, we know:
- how likely the most probable path is, and  
- what sequence of hidden states produced it.

---

### Implementation heuristic

In the code version:
- `viterbi_probs[t, s]` corresponds to $\delta_t(s)$  
- `backpointer[t, s]` corresponds to $\psi_t(s)$  
- We first fill these matrices recursively (forward phase)  
- Then reconstruct the best path by following the indices backward (backtracking phase)

This mirrors the mathematical principle:

> “The probability of the most probable path ending in state $s_t$ with observation $o_t$ is the emission probability of $o_t$ in $s_t$, times the maximum over all possible previous states of (the probability of the best path ending in that previous state × transition probability).”


In [1]:
import numpy as np

#HMM parameters
states = ["Sunny", "Rainy"]
observations = ["Walk", "Shop", "Clean"]

transition_probs = np.array([
    [0.8, 0.2],  # from Sunny -> Sunny / Rainy
    [0.4, 0.6]   # from Rainy -> Sunny / Rainy
])

emission_probs = np.array([
    [0.6, 0.3, 0.1],  # if Sunny: Walk / Shop / Clean
    [0.1, 0.4, 0.5]   # if Rainy: Walk / Shop / Clean
])

initial_probs = np.array([0.7, 0.3])

observed_sequence = ["Walk", "Shop", "Clean"]

obs_to_idx = {obs: idx for idx, obs in enumerate(observations)}


In [4]:
def viterbi_algorithm(observed_seq, states, initial_probs, transition_probs, emission_probs):
    num_states = len(states)
    num_obs = len(observed_seq)

    # Probability table
    viterbi_probs = np.zeros((num_obs, num_states))
    
    # Backpointer table (to reconstruct the best path)
    backpointer = np.zeros((num_obs, num_states), dtype=int)

    # --- Initialization ---
    for s in range(num_states):
        viterbi_probs[0, s] = initial_probs[s] * emission_probs[s, obs_to_idx[observed_seq[0]]]
        backpointer[0, s] = 0

    # --- Recursion ---
    for t in range(1, num_obs):
        for s in range(num_states):
            # For each current state s, find the previous state that maximizes the probability
            prob_transition = [
                viterbi_probs[t-1, prev_s] * transition_probs[prev_s, s]
                for prev_s in range(num_states)
            ]
            best_prev_state = np.argmax(prob_transition)
            viterbi_probs[t, s] = prob_transition[best_prev_state] * emission_probs[s, obs_to_idx[observed_seq[t]]]
            backpointer[t, s] = best_prev_state

    # --- Termination ---
    best_last_state = np.argmax(viterbi_probs[-1, :])
    best_path_prob = viterbi_probs[-1, best_last_state]

    # --- Path backtracking ---
    best_path = [best_last_state]
    print(best_path)
    for t in range(num_obs - 1, 0, -1):
        best_path.insert(0, backpointer[t, best_path[0]])

        print(t, best_path)
    best_state_sequence = [states[i] for i in best_path]

    return viterbi_probs, backpointer, best_state_sequence, best_path_prob


In [8]:
viterbi_probs, backpointer, best_path, best_prob = viterbi_algorithm(
    observed_sequence, states, initial_probs, transition_probs, emission_probs
)

print("Viterbi probability matrix:")
print(viterbi_probs)

print("backpointer")
print(backpointer)

print("\nMost likely hidden state sequence:")
print(best_path)

print("\nProbability of this sequence:")
print(best_prob)


[np.int64(1)]
2 [np.int64(0), np.int64(1)]
1 [np.int64(0), np.int64(0), np.int64(1)]
Viterbi probability matrix:
[[0.42     0.03    ]
 [0.1008   0.0336  ]
 [0.008064 0.01008 ]]
backpointer
[[0 0]
 [0 0]
 [0 0]]

Most likely hidden state sequence:
['Sunny', 'Sunny', 'Rainy']

Probability of this sequence:
0.01008
